# Indexing in Azure Cosmos DB

Let's explore how [indexing](https://docs.microsoft.com/en-us/azure/cosmos-db/index-overview) works in Azure Cosmos DB! In this notebook, we will present how to update indexing policies and how indexing can affect query performance.

Let's start by creating the resources we are going to need: a database and a container.

In [ ]:
import os
import time
import azure.cosmos

db_name = "iddbtest"
container_name = "idcltest"

# Create the database if it doesn't exist
try:
    cosmos_client.create_database(db_name)
    print('Database created')
    time.sleep(2)
except azure.cosmos.errors.CosmosHttpResponseError as e:
    if e.status_code == 409:
       print('Database already exists')
    else: 
        raise

# Reset the container
database_client = cosmos_client.get_database_client(db_name)
containers = database_client.list_containers()
if (any(container['id'] == container_name for container in containers)):
    database_client.delete_container(container_name)
    print('Container dropped')
database_client.create_container(container_name, azure.cosmos.PartitionKey(path="/id"))
print('Container created')

### Importing documents

We will need some test data to work with, so we import 10,000 documents. Each document contains 2 fields: `field1` which has a random string value and `field2` which has a random integer value:

```
{
  "field1":"Garry75",
  "field2":405
}
```

This is going to take some time, so feel free to get a coffee ☕ in the meantime!

In [ ]:
import urllib.request, json

# fetch and load the data
with urllib.request.urlopen("https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/indexing.json") as url:
    docs = json.loads(url.read().decode())

# insert the docs
container_client = database_client.get_container_client(container_name)
progress = 1
for doc in docs:
    if progress % 10 == 0:
        print("Inserting document: %5d / 10000\r"% (progress), end="", flush=True)
    container_client.create_item(doc)
    progress += 1

### Testing a first query

Now that we have our test data, time to issue some queries! We start by fetching all documents with a `field1` value of `Mario86`:

In [ ]:
query = 'SELECT * FROM c WHERE c.field1 = "Mario86"'
start = time.time()
results = list(container_client.query_items(
    query=query,
    enable_cross_partition_query=True))
end = time.time()

print('Got ' + str(len(results)) + ' result(s)')
print('Time elapsed: %d ms'% ((end - start) * 1000))
print('Request charge: ' + container_client.client_connection.last_response_headers['x-ms-request-charge'] + ' RUs')

We got 2 results back in a pretty short amount of time (if the elapsed time looks large, just re-run the previous cell a couple of times - Cosmos DB Notebooks are running on a shared infrastructure so time measurements aren't always very precise). Also, we see that this query has consumed a very small amount of [Request Units](https://docs.microsoft.com/en-us/azure/cosmos-db/request-units) (or RUs).
This is because by default, Cosmos DB indexes *all* the fields it finds in the JSON documents you store. This lets you achieve good performance from the start, without the need to think about secondary indexes upfront.

### Disabling the index

So what would be the performance of that same query if there was no index? To find out, we can completely disable indexing by updating our container's [indexing policy](https://docs.microsoft.com/en-us/azure/cosmos-db/index-policy). Specifically, we set the policy's indexing mode to `none`:

In [ ]:
database_client.replace_container(container_client,
    partition_key=azure.cosmos.PartitionKey(path="/id"),
    indexing_policy={
        'indexingMode': 'none'
    })
print('Indexing policy updated')

Now let's run the same query again and see the results.

In [ ]:
query = 'SELECT * FROM c WHERE c.field1 = "Mario86"'
start = time.time()
results = list(container_client.query_items(
    query=query,
    enable_cross_partition_query=True))
end = time.time()

print('Got ' + str(len(results)) + ' result(s)')
print('Time elapsed: %d ms'% ((end - start) * 1000))
print('Request charge: ' + container_client.client_connection.last_response_headers['x-ms-request-charge'] + ' RUs')

A pretty big difference, both in terms of latency and RUs consumed!

So indexes are very useful to improve the performance and cost-effectiveness of a data model, and it's great that Cosmos DB indexes every property by default.

But in some situations, you may want to fine-tune the default indexing policy by explicitly removing from the index the properties that you won't filter on in your queries. This optimization yields 2 kinds of benefits:

- it will reduce the amount of storage consumed by your container
- it will reduce the latency and RU consumption of write operations

### Excluding paths from the index

Let's do that by updating our indexing policy again, setting the indexing mode back to `consistent` (which is the default) and including all property paths except `field2` (see [this page](https://docs.microsoft.com/en-us/azure/cosmos-db/index-policy#including-and-excluding-property-paths) for a detailed explanation of indexing path syntax).

When an indexing policy is updated with an indexing mode set to `consistent`, Cosmos DB starts to rebuild the index asynchronously. We can monitor the progress of this operation by reading the corresponding container and fetch the transformation progress from a specific response header.

In [ ]:
database_client.replace_container(container_client,
    partition_key=azure.cosmos.PartitionKey(path="/id"),
    indexing_policy={
        'indexingMode': 'consistent',
        'includedPaths': [{'path' : '/*'}],
        'excludedPaths': [{'path' : '/field2/?'}]
    })
print('Indexing policy updated')

container_client.read(populate_quota_info=True)
index_transformation_progress = container_client.client_connection.last_response_headers['x-ms-documentdb-collection-index-transformation-progress']
print('\rCurrent index transformation progress: ' + index_transformation_progress + '%', end="", flush=True)
while (index_transformation_progress != '100'):
    time.sleep(5)
    container_client.read(populate_quota_info=True)
    index_transformation_progress = container_client.client_connection.last_response_headers['x-ms-documentdb-collection-index-transformation-progress']
    print('\rCurrent index transformation progress: ' + index_transformation_progress + '%', end="", flush=True)
print('\nIndex transformation completed')

We now have an indexing policy that indexes everything except the `field2` property. Let's verify that with some queries!

Our previous query filtering on `field1` gets its original performance back:

In [ ]:
query = 'SELECT * FROM c WHERE c.field1 = "Mario86"'
start = time.time()
results = list(container_client.query_items(
    query=query,
    enable_cross_partition_query=True))
end = time.time()

print('Got ' + str(len(results)) + ' result(s)')
print('Time elapsed: %d ms'% ((end - start) * 1000))
print('Request charge: ' + container_client.client_connection.last_response_headers['x-ms-request-charge'] + ' RUs')

But a query that filters on `field2` won't benefit from the index and yield poor performance:

In [ ]:
query = 'SELECT * FROM c WHERE c.field2 = 3188'
start = time.time()
results = list(container_client.query_items(
    query=query,
    enable_cross_partition_query=True))
end = time.time()

print('Got ' + str(len(results)) + ' result(s)')
print('Time elapsed: %d ms'% ((end - start) * 1000))
print('Request charge: ' + container_client.client_connection.last_response_headers['x-ms-request-charge'] + ' RUs')

Check [this page](https://docs.microsoft.com/en-us/azure/cosmos-db/how-to-manage-indexing-policy) to explore the different ways to manage indexing policies, including the Azure Portal, Azure CLI, or any Cosmos DB SDK.

And before we close, don't forget to clean up the resources we've created:

In [ ]:
cosmos_client.delete_database(db_name)